# Preprocess entire and then use existing event_samples to create Epochs anew

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils_epo as pre_utils_epo 
from autoreject import AutoReject

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)
RANDOM_STATE = config.parameters.random_seed

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

event_id = configs.CFGLog['parameters']['event_id']
tmin, tmax = config.parameters.tmin, config.parameters.tmax

imported src module
imported preprocessing module
imported utils module
imported configs module
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}


In [2]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        
        subject.preprocess()
        
        # Load epochs info
        subject.load_epochs_info()

        # Create epochs from events
        subject.epochs = pre_utils_epo.create_epochs_from_events(
            subject.preprocessed_raw, 
            sub_id, 
            subject.events, 
            event_id, 
            tmin, 
            tmax)
        
        # Reject and update epochs using AutoReject
        subject.reject_and_update_epochs()

        # Save
        subject.save(subject.epochs, "epochs")
        subject.save(subject.epochs.get_data(), "epochs", as_mat=True)
        
        # Concatenate epochs into VHDR for visualization
        subject.concatenate_epochs(save=True, overwrite=True)

Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA/Acquisition 018.Part2 Data.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2959599  =      0.000 ...  2959.599 secs...
Loaded raw for subject 018
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.8s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 58 - 62 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 58.00
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 57.75 Hz)
- Upper passband edge: 62.00 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 62.25 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 32 components
